In [3]:
%load_ext tensorboard
%load_ext dotenv

%dotenv

In [2]:
import os
import openai
from pydantic import BaseModel

MODEL = "gpt-4o-mini"

openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
x="Mathematical Theorems and Proofs would formalize the relationships between mathematical concepts, theorems, proofs, axioms, and logical inference rules."

In [5]:
from typing import Literal

from pydantic import Field

# note: maybe a different prompt would work better, i.e. name people that could be involved with the topic, then ask to generate them, getting a more diverse list potentially?

generate_personas_prompt = """You are an ontology engineer tasked with creating an ontology on <topic>{topic}</topic>. Find {count} individuals who you would like to interview to figure out initial competency questions. Ensure a diverse mix of backgrounds, experiences, and viewpoints. Include people from different professions, levels of education, socioeconomic statuses, geographical locations, and lived experiences. Focus on real-world perspectives from all walks of life. The goal is to avoid over-representing a single demographic and instead capture a broad spectrum of voices that could contribute valuable insights on this topic."""


class Education(BaseModel):
    institution: str
    degree: str | None
    field_of_study: str | None
    description: str | None

class WorkExperience(BaseModel):
    company: str
    position: str
    description: str | None

class Skill(BaseModel):
    name: str
    proficiency: Literal["beginner", "intermediate", "advanced"] | None

class Persona(BaseModel):
    name: str
    bio: str = Field(description="Short bio written by the person themselves. (\"I am ...\")")
    education: list[Education] | None
    work_experience: list[WorkExperience] | None
    skills: list[Skill] | None



class Personas(BaseModel):
    items: list[Persona]

def generate_personas(topic: str, count: int):
    prompt = generate_personas_prompt.format(count=count, topic=topic)

    response = openai.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt}
        ],
        response_format=Personas
    )

    # todo make sure we have the right number of personas, else generate more

    return response.choices[0].message.parsed

personas = generate_personas(x, 10)

print(len(personas.items))
print(personas.model_dump_json(indent=True))

10
{
 "items": [
  {
   "name": "Alice Thompson",
   "bio": "I am a high school mathematics teacher passionate about making math accessible to all students.",
   "education": [
    {
     "institution": "State University",
     "degree": "Bachelor's",
     "field_of_study": "Mathematics Education",
     "description": ""
    },
    {
     "institution": "State University",
     "degree": "Master's",
     "field_of_study": "Curriculum Development",
     "description": ""
    }
   ],
   "work_experience": [
    {
     "company": "Lincoln High School",
     "position": "Mathematics Teacher",
     "description": "Teaching algebra, geometry, and calculus to high school students."
    },
    {
     "company": "Math Tutoring Center",
     "position": "Part-Time Tutor",
     "description": "Providing one-on-one tutoring to students struggling with math."
    }
   ],
   "skills": [
    {
     "name": "Mathematical Proofs",
     "proficiency": "advanced"
    },
    {
     "name": "Curriculum Des

In [ ]:
from typing import Any


expert_system_prompt = """# Role & Objective

You are {description}, an expert in {topic}. You are being interviewed by an ontology engineer who is developing an ontology for this domain.

Your role is to share your domain knowledge clearly and concisely. You do not need to know about ontologies or competency questions—that is the ontology engineer’s responsibility.

# Guidelines for the Conversation

- Keep your answers concise and to the point.
- Do not repeat information that has already been discussed, unless you need to clarify something.
- Ask for clarification if needed, but focus on providing structured insights from your expertise.
- Answer questions based on your domain knowledge, particularly in relation to:
  - Entities: What are the core concepts in your field? What are their essential characteristics?
  - Relationships: How do these concepts connect? Are there hierarchies, dependencies, or causal links?
  - Rules & Constraints: What rules, business logic, or restrictions exist in the domain?
  - Processes & Events: What key workflows, actions, or transformations occur? What triggers them?
  - Disambiguation: Are there terms that are often confused or used differently by different people?
  - Context & Scope: What should be included or excluded when modeling this domain?

# What You Are Not Responsible For

- You do not need to worry about ontologies, competency questions, or formal structuring—the ontology engineer will handle that.
- You do not need to repeat yourself unless clarification is required.

This should be a structured yet natural conversation where you share your expertise while the ontology engineer extracts key insights."""

engineer_system_prompt = """# Role & Objective

You are an ontology engineer developing an ontology on {topic}. You are interviewing {description}, an expert in this domain, to gather structured knowledge.

Your goal is to extract key domain concepts, relationships, rules, and constraints to inform ontology design. You will also define competency questions—formal questions that the ontology should be able to answer—to shape its structure.

# Guidelines for the Conversation

- Ask concise, focused questions.
- Do not repeat what the expert has already said, except when referring back for clarification.
- Follow up on responses to refine understanding and resolve ambiguities.
- Do not ask about ontologies or competency questions directly—translating expert knowledge into ontology structures is your responsibility.
- Your questions should focus on key aspects of ontology design, including:
  - Entities: What are the core concepts in this domain? What are their essential attributes?
  - Relationships: How do these entities connect? Are relationships hierarchical, causal, or associative?
  - Rules & Constraints: What logical dependencies, business rules, or constraints exist?
  - Processes & Events: What workflows, actions, or transformations occur? What triggers them?
  - Disambiguation: Are there any commonly confused terms or overlapping concepts?
  - Context & Scope: What should be included or excluded in this ontology?

# Competency Questions

- Based on the expert’s answers, you will formulate competency questions (i.e., queries that the ontology must be able to answer).
- Do not ask competency questions as direct questions to the expert—instead, extract the necessary information to define them yourself.
- Example: If the expert explains relationships between disease symptoms and treatments, a competency question could be: "Given a set of symptoms, what possible diseases could be diagnosed?"
- Use competency questions to guide ontology design, not as conversation prompts.

Your role is to structure knowledge efficiently while keeping the conversation natural and to the point."""

Speaker = Literal["expert", "engineer"]

class Turn(BaseModel):
    """A turn in the conversation between the expert and the engineer"""

    role: Speaker
    content: str

    def to_openai_format(self, speaker: Speaker) -> Any:
        return {"role": ("assistant" if self.role == speaker else "user"), "content": self.content}

class Interview(BaseModel):
    persona: Persona
    topic: str
    protocol: list[Turn]

    def add_turn(self, role: Speaker, content: str):
        self.protocol.append(Turn(role=role, content=content))


class CompetencyQuestion(BaseModel):
    question: str
    confidence: float = Field(description="Your confidence in this question on a scale from 0 to 1. Should only increase with facts!")

    notes: str | None = Field(description="Any notes or context that might be relevant to this question, especially if you are not sure this works. Maybe also just ask the expert!")

# end conversation action after predefined number of turns, time limit or just provide in general?


class DefineCompetencyQuestionAction(BaseModel):
    """Define a competency question"""
    
    type: Literal["def_cq"]
    cq: CompetencyQuestion

class Thought(BaseModel):
    chain_of_thought: str = Field(description="Your INTERNAL MONOLOGUE. Think about what was just said and what you should do next!")

    actions: list[DefineCompetencyQuestionAction]

    response: str = Field(description="Your response to the expert. Keep it as concise as possible.")

class Memory(BaseModel):
    questions: list[CompetencyQuestion]
    thoughts: list[Thought]




def expert_respond(interview: Interview):
    """Simulate the expert's response to the engineer's """

    
    sys_prompt = expert_system_prompt.format(description=interview.persona.model_dump_json(), topic=interview.topic)


    response = openai.chat.completions.create(
            model=MODEL,
            messages=[{"role": "system", "content": sys_prompt},
                      *[turn.to_openai_format("expert") for turn in interview.protocol]
            ]
        )
    
    return response.choices[0].message.content

def engineer_think(interview: Interview):
    """Simulate the engineer's thought process after the expert's response"""

    sys_prompt = engineer_system_prompt.format(description="You are an ontology engineer ", topic=interview.topic)

    response = openai.beta.chat.completions.parse(
            model=MODEL,
            messages=[{"role": "system", "content": sys_prompt},
                      *[turn.to_openai_format("engineer") for turn in interview.protocol]
            ],
            response_format=Thought
        )

    return response.choices[0].message.parsed

def interview(persona: Persona, topic: str):
    """A simulated interview between a domain expert and an ontology engineer"""

    interview = Interview(persona=persona, topic=topic, protocol=[
        Turn(role="engineer", content="Thank you for taking the time to speak with me. My goal today is to better understand your domain and how knowledge is structured within it. There's no fixed script, so I'd love to hear your thoughts and experiences naturally. Can you describe your field and what kind of problems you typically deal with?"),
    ])

    memory = Memory(questions=[], thoughts=[])

    while True:
        response = expert_respond(interview)

        if not response:
            raise Exception("No response from expert, something weird happened")

        interview.add_turn("expert", response)

        print(f"{persona.name}: {response}")

        thought = engineer_think(interview)

        if not thought:
            raise Exception("No response from engineer, something weird happened")
        
        interview.add_turn("engineer", thought.response)
        
        memory.thoughts.append(thought)
        # append new cqs
        for action in thought.actions:
            if isinstance(action, DefineCompetencyQuestionAction):
                memory.questions.append(action.cq)

        print(thought.model_dump_json(indent=True))

        print(f"Engineer: {thought.response}")


interview(personas.items[0], x)




Alice Thompson: Thank you for having me. My field primarily revolves around high school mathematics, where I teach various topics such as algebra, geometry, and calculus. The core focus is on helping students understand mathematical concepts, develop critical thinking skills, and apply those concepts to solve problems.

Typically, the problems I encounter involve:

1. **Understanding Concepts**: Students often struggle with grasping foundational concepts, which leads to difficulties in more advanced topics. My role is to clarify these concepts and connect them to practical applications.

2. **Logical Reasoning**: Teaching students how to construct logical arguments and proofs is essential, particularly in geometry and higher math. This involves illustrating the relationships between axioms, theorems, and logical inference.

3. **Curriculum Development**: I continuously assess and adjust our curriculum to ensure it meets students' needs while adhering to educational standards. This can 